# Design of a BASIC linker for the backbone collection

This notebook contains code relating to the design of a linker to join parts needed to generate BASIC DNA assembly backbones.

## Aims and objectives for cells below

- Generate a unique job ID for the r2o job specification.

In [18]:
import hashlib
import basicsynbio as bsb

TypeError: 'type' object is not subscriptable

In [10]:
print(hashlib.md5("basic_bb_linker_001".encode("UTF-8")).hexdigest())


92f0817faf716045872b354b5b2482a7


## Aims and objectives for cells below

- Generate sequence for specification file.

## Notes of sequence generation

- The original neutral linker designs were 45 bp in length. To this the necessary 8 bases required for ligation to BsaI digested parts were added, yielding 53 bp sequences.



In [12]:
format = "GGCTCG" + 45*"N" + "GTCC"
print("Format " + format)
assert len(format) == 53 + 2

Format GGCTCGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTCC


## Notes on parameter generation

- Each prefix and suffix half contained in addition to required homology to BsaI-digested parts a 12 bp double stranded region and a 21 base ss overlap region driving annealing between linker-ligated-parts. The target Tm for the ss region was set to 60 °C.
- The overall target GC content of the linker was set to 40 %, similar to the original linker designs.
- Some forbidden sequences:
    - BsaI restriction site: GGTCTC
    - Prefix 4 bp overhang and rc: GTCC & rc
    - Suffix 4 bp overhang and rc: CTCG & rc
    - Various E. coli promoter sites: (refer to r2o job spec file)
    - G-quadruplex: GGGG
    - Translation initiation site: ATG
    - RBS core: AAAGA
    - SD sequence: AGGAGG
    - IS10 site: NGCTNAGCN
    - IS231 ste: GGGNNNNNCCC
    - More detailed in the specification job file.

## Aims and objectives for cell/s below

- Export key sequences from basicsynbio as a FASTA file. The backbone linker should have low homology to these sequences.

In [17]:
collections_to_avoid = [
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"],
    bsb.BASIC_PROMOTER_PARTS["v0.2"]
]
sequences = [value for collection in collections_to_avoid for value in collection.values()]



NameError: name 'bsb' is not defined